In [508]:
from lie_learn.spectral.SE2FFT import SE2_FFT, shift_fft, shift_ifft

In [258]:
# from lie_learn.spectral.SE2FFT import map_wrap

In [418]:
from lie_learn.spectral.T2FFT import T2FFT

In [419]:
from lie_learn.spectral.T1FFT import T1FFT

In [70]:
import numpy as np
from scipy.stats import multivariate_normal
from typing import Tuple

In [72]:
from numpy.fft import ifftshift, fftshift
from einops import rearrange

In [71]:
import math

In [448]:
cov_np = np.diag([0.1, 0.1, 0.1])
# cov = np.eye(3) * 1e-4
# mu = np.array([0.1,0.1,np.pi/2])
mu_np = np.array([ 0.  , 0.15,  0.  ])
def se2_grid_samples(size: Tuple[int] = (5, 5, 5),
                     lower_bound: float = 0,
                     upper_bound: float = 1) -> np.ndarray:
    xs = np.linspace(lower_bound, upper_bound, size[0], endpoint=False)
    ys = np.linspace(lower_bound, upper_bound, size[1], endpoint=False)
    ts = np.linspace(0., 2. * np.pi, size[2], endpoint=False)
    X, Y, T = np.meshgrid(xs, ys, ts, indexing='ij')
    poses = np.vstack((X.flatten(), Y.flatten(), T.flatten())).T
    return poses, X, Y, T

grid_size = 
poses_np, _, _, _ = se2_grid_samples(grid_size)
diff = poses_np - mu_np
# Wrap angle
diff[:, 2] = (diff[:, 2] + np.pi) % (2 * np.pi) - np.pi
energy_np = multivariate_normal.logpdf(diff, mean=np.zeros(3), cov=cov_np)
energy_np = energy_np.reshape(grid_size)

In [13]:
d = cov_np.shape[0]  # Dimensionality of the Gaussian distribution
det_cov = np.linalg.det(cov_np)  # Determinant of the covariance matrix

# Normalization constant formula
constant = np.log((2 * np.pi) ** (d / 2) * np.sqrt(det_cov))

In [491]:
fft_np = SE2_FFT(spatial_grid_size=(50,50,32),
                      interpolation_method='spline',
                      spline_order=1,
                      oversampling_factor=1)

In [509]:
energy_torch_np = energy.numpy()

In [362]:
def compute_moments_lnz(energy, update: bool = True) -> Tuple[np.ndarray, float]:
        """
        Compute the moments of the distribution
        :param eta: canonical parameters in log prob space of distribution
        :param update: whether to update the moments of the distribution and log partition constant
        :return: moments and log partition constant
        """
        
        _, _, _, _, _, eta = fft_np.analyze(energy)
        negative_energy, _, _, _, _, _ = fft_np.synthesize(eta)
        maximum = np.max(negative_energy)
        _, _, _, _, _, unnormalized_moments = fft_np.analyze(np.exp(negative_energy - maximum))
        # TODO: Figure out why z_0 is the 0 index.
        z_0 = 0
        z_1 = unnormalized_moments.shape[1] // 2
        z_2 = unnormalized_moments.shape[2] // 2
        # Scale by invariant haar measure
        # Haar measure in Chirikjian's book
        # unnormalized_moments[z_0, z_1, z_2] *= np.power(2 * np.pi, 2)
        # Haar measure in Chirikjian's book for S1, semi-direct product R^2 \cross S1
        unnormalized_moments[z_0, z_1, z_2] *= np.pi * 2
        moments = unnormalized_moments / unnormalized_moments[z_0, z_1, z_2]
        moments[z_0, z_1, z_2] = unnormalized_moments[z_0, z_1, z_2]
        l_n_z = np.log(unnormalized_moments[z_0, z_1, z_2]) + maximum
        # Update moments of distribution and constant only when needed
        return moments, l_n_z.real

In [510]:
f, f1c, f1p, f2, f2f, fh = fft_np.analyze(energy_torch_np)

In [511]:
f_1, f1c_1, f1p_1, f2_1, f2f_1, fh_1 = fft_np.synthesize(fh)

In [512]:
np.mean(abs(energy_torch_np - f_1))

0.43993245008704596

In [226]:
f_1_d_np, f1c_1_d_np, f1p_1_d_np, f2_1_d_np, f2f_1_d_np, fh_1_d_np

(array([[[-0.02687275-1.70768574e-02j, -0.02687276-1.70768723e-02j,
          -0.02687278-1.70768835e-02j, ..., -0.02687277-1.70768965e-02j,
          -0.02687277-1.70768350e-02j, -0.02687278-1.70768760e-02j],
         [ 0.01757718-7.00317174e-02j,  0.01757721-7.00317249e-02j,
           0.01757722-7.00317025e-02j, ...,  0.01757719-7.00317547e-02j,
           0.0175772 -7.00316951e-02j,  0.01757719-7.00317398e-02j],
         [ 0.03574726+2.58313529e-02j,  0.03574728+2.58313231e-02j,
           0.03574724+2.58313324e-02j, ...,  0.03574724+2.58312523e-02j,
           0.03574728+2.58313306e-02j,  0.03574727+2.58313157e-02j],
         ...,
         [ 0.00526831-3.23792063e-02j,  0.00526829-3.23792286e-02j,
           0.00526829-3.23791988e-02j, ...,  0.00526826-3.23792547e-02j,
           0.00526828-3.23792323e-02j,  0.00526834-3.23792249e-02j],
         [ 0.03480201+9.11138393e-03j,  0.03480199+9.11140069e-03j,
           0.03480199+9.11145471e-03j, ...,  0.03480197+9.11142677e-03j,
     

In [424]:
f1c_check = fft_np.resample_p2c_3d(f1c_1)
# f_check = shift_ifft(f1c_check)

In [425]:
f1c_check.shape

(10, 5, 32)

In [286]:
import numpy as np

def bilinear_interpolate_with_nan_safe(f, x, y, mode='constant'):
    """
    Perform bilinear interpolation on a 2D NumPy array with `NaN` padding,
    safely handling `NaN` values during the computation.

    Parameters:
        f (np.ndarray): Input 2D array of shape (H, W).
        x (np.ndarray): x-coordinates to interpolate (float array).
        y (np.ndarray): y-coordinates to interpolate (float array).

    Returns:
        np.ndarray: Interpolated values at the given coordinates.
    """
    # Pad the array with NaN
    if mode == "constant":
        f = np.pad(f, ((1, 1), (1, 1)), mode, constant_values=np.nan)
    elif mode == "wrap":
        f = np.pad(f, ((1, 1), (1, 1)), mode)

    # Adjust coordinates for the padded array
    x = np.asarray(x) + 1
    y = np.asarray(y) + 1

    # Get integer grid points
    x0 = np.floor(x).astype(int)
    x1 = x0 + 1
    y0 = np.floor(y).astype(int)
    y1 = y0 + 1

    # Clip to ensure indices are within bounds
    x0 = np.clip(x0, 0, f.shape[1] - 1)
    x1 = np.clip(x1, 0, f.shape[1] - 1)
    y0 = np.clip(y0, 0, f.shape[0] - 1)
    y1 = np.clip(y1, 0, f.shape[0] - 1)

    # Gather values from the 2D array
    Ia = f[x0, y0]  # Bottom-left
    Ib = f[x1, y0]  # Top-left
    Ic = f[x0, y1]  # Bottom-right
    Id = f[x1, y1]  # Top-right

    # Compute the weights for interpolation
    wa = (x1 - x) * (y1 - y)  # Weight for Ia
    wb = (x1 - x) * (y - y0)  # Weight for Ib
    wc = (x - x0) * (y1 - y)  # Weight for Ic
    wd = (x - x0) * (y - y0)  # Weight for Id

    # Mask NaN values
    valid_a = ~np.isnan(Ia)
    valid_b = ~np.isnan(Ib)
    valid_c = ~np.isnan(Ic)
    valid_d = ~np.isnan(Id)

    # Set invalid contributions to zero
    Ia = np.where(valid_a, Ia, 0)
    Ib = np.where(valid_b, Ib, 0)
    Ic = np.where(valid_c, Ic, 0)
    Id = np.where(valid_d, Id, 0)

    # Adjust weights for NaN values
    wa = wa * valid_a
    wb = wb * valid_b
    wc = wc * valid_c
    wd = wd * valid_d

    # Normalize weights to handle missing data
    weight_sum = wa + wb + wc + wd
#     normalized_wa = wa / (weight_sum + 1e-8)
#     normalized_wb = wb / (weight_sum + 1e-8)
#     normalized_wc = wc / (weight_sum + 1e-8)
#     normalized_wd = wd / (weight_sum + 1e-8)

    # Compute the interpolated value
    result = (
        wa * Ia +
        wb * Ib +
        wc * Ic +
        wd * Id
    )

    # If all weights are zero, set result to NaN
#     result[weight_sum == 0] = np.nan

    return result


In [287]:
def map_wrap(f, coords, mode):

    # Create an agumented array, where the last row and column are added at the beginning of the axes
    fa = np.empty((f.shape[0] + 1, f.shape[1] + 1))
    #fa[1:, 1:] = f
    #fa[0, 1:] = f[-1, :]
    #fa[1:, 0] = f[:, -1]
    #f[0, 0] = f[-1, -1]
    fa[:-1, :-1] = f
    fa[-1, :-1] = f[0, :]
    fa[:-1, -1] = f[:, 0]
    fa[-1, -1] = f[0, 0]

    # Wrap coordinates
    wrapped_coords_x = coords[0, ...] % f.shape[0]
    wrapped_coords_y = coords[1, ...] % f.shape[1]
#     wrapped_coords = np.r_[wrapped_coords_x[None, ...], wrapped_coords_y[None, ...]]

    # Interpolate
    #return fa, wrapped_coords, map_coordinates(f, wrapped_coords, order=1, mode='constant', cval=np.nan, prefilter=False)
    return bilinear_interpolate_with_nan_safe(fa, wrapped_coords_x, wrapped_coords_y,mode)


In [304]:
class SE2_FFT_NP():

    def __init__(self,
                 spatial_grid_size=(10, 10, 10),
                 interpolation_method='spline',
                 spline_order=1,
                 oversampling_factor=1):

        self.spatial_grid_size = spatial_grid_size  # tau_x, tau_y, theta
        self.interpolation_method = interpolation_method

        if interpolation_method == 'spline':
            self.spline_order = spline_order

            # The array coordinates of the zero-frequency component
            self.p0 = spatial_grid_size[0] // 2
            self.q0 = spatial_grid_size[1] // 2

            # The distance, in pixels, from the (0, 0) pixel to the center of frequency space
            self.r_max = np.sqrt(self.p0 ** 2 + self.q0 ** 2)

            # Precomputation for cartesian-to-polar regridding
            self.n_samples_r = int(oversampling_factor * (np.ceil(self.r_max) + 1))
            self.n_samples_t = int(oversampling_factor * (np.ceil(2 * np.pi * self.r_max)))

            r = np.linspace(0., self.r_max, self.n_samples_r, endpoint=True)
            theta = np.linspace(0, 2 * np.pi, self.n_samples_t, endpoint=False)
            R, THETA, = np.meshgrid(r, theta, indexing='ij')

            # Convert polar to Cartesian coordinates
            X = R * np.cos(THETA)
            Y = R * np.sin(THETA)

            # Transform to array indices (note; these are not the usual coordinates where y axis is flipped)
            I = X + self.p0
            J = Y + self.q0

            self.c2p_coords = np.r_[I[None, ...], J[None, ...]]


            # Precomputation for polar-to-cartesian regridding
            i = np.arange(0, self.spatial_grid_size[0])
            j = np.arange(0, self.spatial_grid_size[1])
            x = i - self.p0
            y = j - self.q0
            X, Y = np.meshgrid(x, y, indexing='ij')

            # Convert Cartesian to polar coordinates:
            R = np.sqrt(X ** 2 + Y ** 2)
            T = np.arctan2(Y, X) #  % (2 * np.pi)

            # Convert to array indices
            # Maximum of R is r_max, maximum index in array is (n_samples_r - 1)
            R *= (self.n_samples_r - 1) / self.r_max
            # The maximum angle in T is arbitrarily close to 2 pi,
            # but this should end up 1 pixel past the last index n_samples_t - 1, i.e. it should end up at n_samples_t
            # which is equal to index 0 since wraparound is used.
            T *= self.n_samples_t / (2 * np.pi)

            self.p2c_coords = np.r_[R[None, ...], T[None, ...]]
        elif interpolation_method == 'Fourier':

            #r_max = np.sqrt(2)
            r_max = 1. / np.sqrt(2.)
            #nr = spatial_grid_size[0] + 1
            nr = 15 * np.ceil(r_max * spatial_grid_size[0])
            nt = 5 * np.ceil(2 * np.pi * r_max * spatial_grid_size[0])
            nx = spatial_grid_size[0]
            ny = spatial_grid_size[1]
            self.flerp = FourierInterpolator.init_cartesian_to_polar(nr, nt, nx, ny)

        else:
            raise ValueError('Unknown interpolation method:' + str(interpolation_method))

    def analyze(self, f):
        """
        Compute the SE(2) Fourier Transform of a function f : SE(2) -> C or f : SE(2) -> R.
        The SE(2) Fourier Transform expands f in the basis of matrix elements of irreducible representations of SE(2).
        Let T^r_pq(g) be the (p, q) matrix element of the irreducible representation of SE(2) of weight / radius r,
        then the FT is:

        F^r_pq = int_SE(2) f(g) conjugate(T^r_pq(g^{-1})) dg

        We assume g in SE(2) to be parameterized as g = (tau_x, tau_y, theta), where tau is a 2D translation vector
        and theta is a rotation angle.
        The input f is a 3D array of shape (N_x, N_y, N_t),
        where the axes correspond to tau_x, tau_y, theta in the ranges:
        tau_x in np.linspace(-0.5, 0.5, N_x, endpoint=False)
        tau_y in np.linspace(-0.5, 0.5, N_y, endpoint=False)
        theta in np.linspace(0, 2 * np.pi, N_t, endpoint=False)

        See:
        "Engineering Applications of Noncommutative Harmonic Analysis", section 11.2
        Chrikjian & Kyatkin

        "The Mackey Machine: a user manual"
        Taco S. Cohen

        :param f: discretely sampled function on SE(2).
         The first two axes of f correspond to translation parameters tau_x, tau_y, and the third axis corresponds to
         rotation angle theta.
        :return: F, the SE(2) Fourier Transform of f. Axes of F are (r, p, q)
        """

        # First, FFT along translation parameters tau_1 and tau_2
        #f1c_shift = T2FFT.analyze(f, axes=(0, 1))
        # This gives: f1c_shift[xi_1, xi_2, theta]
        # where xi_1 and xi_2 are Cartesian (c) coordinates of the frequency domain.
        # However, this is the FT of the *shifted* function on [0, 1), so shift the coefficient back:
        #delta = -0.5  # we're shifting from [0, 1) to [-0.5, 0.5)
        #xi1 = np.arange(-np.floor(f1c_shift.shape[0] / 2.), np.ceil(f1c_shift.shape[0] / 2.))
        #xi2 = np.arange(-np.floor(f1c_shift.shape[1] / 2.), np.ceil(f1c_shift.shape[1] / 2.))
        #XI1, XI2 = np.meshgrid(xi1, xi2, indexing='ij')
        #phase = np.exp(-2 * np.pi * 1j * delta * (XI1 + XI2))
        #f1c = f1c_shift * phase[:, :, None]

        f1c = shift_fft(f)

        # Change from Cartesian (c) to a polar (p) grid:
        f1p = self.resample_c2p_3d(f1c)
        # This gives f1p[r, varphi, theta]

        # FFT along rotation angle theta
        # We conjugate the argument and the ouput so that the complex exponential has positive instead of negative sign
        f2 = T1FFT.analyze(f1p.conj(), axis=2).conj()
        # This gives f2[r, varphi, q]
        # where q ranges from q = -floor(f1p.shape[2] / 2) to q = ceil(f1p.shape[2] / 2) - 1  (inclusive)

        # Multiply f2 by a (varphi, q)-dependent phase factor:
        m_min = -np.floor(f2.shape[2] / 2.)
        m_max = np.ceil(f2.shape[2] / 2.) - 1
        varphi = np.linspace(0, 2 * np.pi, f2.shape[1], endpoint=False)  # may not need this many points on every circle
        factor = np.exp(-1j * varphi[None, :, None] * np.arange(m_min, m_max + 1)[None, None, :])
        f2f = f2 * factor

        # FFT along polar coordinate of frequency domain
        f_hat = T1FFT.analyze(f2f.conj(), axis=1).conj()
        # This gives f_hat[r, p, q]

        return f, f1c, f1p, f2, f2f, f_hat

    def synthesize(self, f_hat):

        f2f = T1FFT.synthesize(f_hat.conj(), axis=1).conj()

        # Multiply f_2 by a phase factor:
        m_min = -np.floor(f2f.shape[2] / 2)
        m_max = np.ceil(f2f.shape[2] / 2) - 1
        psi = np.linspace(0, 2 * np.pi, f2f.shape[1], endpoint=False)  # may not need this many points on every circle
        factor = np.exp(1j * psi[:, None] * np.arange(m_min, m_max + 1)[None, :])

        f2 = f2f * factor[None, ...]

        f1p = T1FFT.synthesize(f2.conj(), axis=2).conj()

        f1c = self.resample_p2c_3d(f1p)


        # delta = -0.5  # we're shifting from [0, 1) to [-0.5, 0.5)
        # xi1 = np.arange(-np.floor(f1c.shape[0] / 2), np.ceil(f1c.shape[0] / 2))
        # xi2 = np.arange(-np.floor(f1c.shape[1] / 2), np.ceil(f1c.shape[1] / 2))
        # XI1, XI2 = np.meshgrid(xi1, xi2, indexing='ij')
        # phase = np.exp(-2 * np.pi * 1j * delta * (XI1 + XI2))
        # f1c_shift = f1c / phase[:, :, None]


        #f = T2FFT.synthesize(f1c, axes=(0, 1))
        #f = T2FFT.synthesize(f1c_shift, axes=(0, 1))
        f = shift_ifft(f1c)


        return f, f1c, f1p, f2, f2f, f_hat

    def resample_c2p(self, fc):
        """
        Resample a function on a Cartesian grid to a polar grid.

        The center of the Cartesian coordinate system is assumed to be in the center of the image at index
        x0 = fc.shape[0] / 2 - 0.5
        y0 = fc.shape[1] / 2 - 0.5
        i.e. for a 2-pixel image, x0 would be at 'index' 2/2-0.5 = 0.5, in between the two pixels.

        The first dimension of the output coresponds to the radius r in [0, r_max=fc.shape[0] / 2. - 0.5]
        while the second dimension corresponds to the angle theta in [0, 2pi).

        :param fc: function values sampled on a Cartesian grid.
        :return: resampled function on a polar grid
        """

        # We are dealing with three coordinate frames:
        # The array indices / image coordinates (i, j) of the input data array.
        # The Cartesian frame (x, y) centered in the image, with the same directions and units (=pixels) on the axes.
        # The polar coordinate frame (r, theta), also centered in the image, with theta=0 corresponding to the x axis.

        # (x0, y0) are the image coordinates / array indices of the center of the Cartesian coordinate frame
        # centered in the image. Note that although they are in the image coordinate frame, they are not necessarily ints.
        #fp_r = map_coordinates(fc.real, self.c2p_coords, order=self.spline_order, mode='wrap')  # 'nearest')
        #fp_c = map_coordinates(fc.imag, self.c2p_coords, order=self.spline_order, mode='wrap')  # 'nearest')
        #fp = fp_r + 1j * fp_c
        fp_r = map_wrap(fc.real, self.c2p_coords, mode="constant")
        fp_c = map_wrap(fc.imag, self.c2p_coords, mode="constant")
        fp = fp_r + 1j * fp_c

        return fp

    def resample_p2c(self, fp): # , order=1, mode='wrap', cval=np.nan):
        fp = fp.unsqueeze(0)
        p2c_coords = self.p2c_coords.unsqueeze(0)
        fc_r = map_wrap(fp.real, p2c_coords, mode="wrap")
        fc_c = map_wrap(fp.imag, p2c_coords, mode="wrap")
        pdb.set_trace()
        fc = fc_r + 1j * fc_c
        return fc

    def resample_c2p_3d(self, fc):

        if self.interpolation_method == 'spline':
            fp = []
            for i in range(fc.shape[2]):
                fp.append(self.resample_c2p(fc[:, :, i]))

            return np.c_[fp].transpose(1, 2, 0)
#             return np.stack(fp,axis=-1)

        elif self.interpolation_method == 'Fourier':

            fp = []
            for i in range(fc.shape[2]):
                fp.append(self.flerp.forward(fc[:, :, i]))

            return np.c_[fp].transpose(1, 2, 0)


    def resample_p2c_3d(self, fp):

        if self.interpolation_method == 'spline':
            fc = []
            for i in range(fp.shape[2]):
                fc.append(self.resample_p2c(fp[:, :, i]))
#             pdb.set_trace()
            return np.c_[fc].transpose(1, 2, 0)
#             return np.stack(fc,axis=-1)

        elif self.interpolation_method == 'Fourier':
            fc = []
            for i in range(fp.shape[2]):
                fc.append(self.flerp.backward(fp[:, :, i]))

            return np.c_[fc].transpose(1, 2, 0)


In [305]:
fft_np_check = SE2_FFT_NP(spatial_grid_size=grid_size,
                      interpolation_method='spline',
                      spline_order=1,
                      oversampling_factor=1)

In [306]:
f, f1c, f1p, f2, f2f, fh = fft_np_check.analyze(energy_torch_np)
f_1, f1c_1, f1p_1, f2_1, f2f_1, fh_1 = fft_np_check.synthesize(fh)

In [307]:
np.mean(abs(f_1 - energy_torch_np))

1705.8780416109569

In [1]:
import os
import sys
import math
base_path = os.path.dirname(os.getcwd())

sys.path.append(base_path)

In [2]:
from src.distributions.SE2.SE2_FFT import SE2_FFT

grid_size = (50,50,32)

fft = SE2_FFT(spatial_grid_size=grid_size,
                      interpolation_method='spline',
                      spline_order=1,
                      oversampling_factor=1)

By wrapping angles to −π,π, you handle periodicity in a way that ensures the shortest distance between two angles is used

In [3]:
import torch

class GaussianSE2:
    def __init__(self, mu, cov, spatial_grid_size):
        """
        Initialize the Gaussian distribution in SE(2).
        
        Args:
            mu (torch.Tensor): Mean vector of shape (3,).
            cov (torch.Tensor): Covariance matrix of shape (3, 3).
            spatial_grid_size (tuple): Size of the spatial grid.
        """
        self.mu = mu
        self.inv_cov = torch.linalg.inv(cov)
        self.det_cov = torch.linalg.det(cov)
        self.normalizing_constant = torch.sqrt((2 * torch.pi) ** 3 * self.det_cov)
        self.spatial_grid_size = spatial_grid_size

    def energy(self, x):
        """
        Compute the energy (log of unnormalized density) for given poses in SE(2).
        
        Args:
            x (torch.Tensor): Input tensor of shape (N, 3), where each row is a pose (x, y, theta).
        
        Returns:
            torch.Tensor: Energy values for each input pose, reshaped to spatial grid size.
        """
        # Compute difference from the mean
        diff = x - self.mu

        # Wrap the angular difference to [-pi, pi]
        diff[:, :, 2] = (diff[:,:, 2] + torch.pi) % torch.pi

        # Compute the energy
        quadratic_form = torch.sum((diff @ self.inv_cov) * diff, dim=-1)
        energy = -0.5 * quadratic_form 
#         - torch.log(self.normalizing_constant)

        # Reshape to spatial grid size
        return energy

    def density(self, x):
        """
        Compute the density for given poses in SE(2).
        
        Args:
            x (torch.Tensor): Input tensor of shape (N, 3), where each row is a pose (x, y, theta).
        
        Returns:
            torch.Tensor: Density values for each input pose, reshaped to spatial grid size.
        """

        # Compute difference from the mean
        diff = x - self.mu

        # Wrap the angular difference to [-pi, pi]
        diff[:, :, 2] = (diff[:, : , 2] + torch.pi) % (torch.pi)

        # Compute the density
        quadratic_form = torch.sum((diff @ self.inv_cov) * diff, dim=-1)
        density = torch.exp(-0.5 * quadratic_form)/self.normalizing_constant

        # Reshape to spatial grid size
        return density


In [151]:
import torch
import math

class HEFilter():
    def __init__(self, band_limit,range_x,range_y):
        super(HEFilter, self).__init__()
        self.band_limit = band_limit
        self.range_x = range_x
        self.range_y = range_y
        self.range_x_diff = self.range_x[1] - self.range_x[0]
        self.range_y_diff = self.range_y[1] - self.range_y[0]

    def pad_for_fft_2d(self, tensor, target_shape):
        pad_h = target_shape[0] - tensor.shape[1]
        pad_w = target_shape[1] - tensor.shape[2]
        # Padding format in PyTorch: (left, right, top, bottom)
        # padded_tensor = torch.nn.functional.pad(tensor, (math.ceil(pad_w/2), pad_w - math.ceil(pad_w/2),math.ceil(pad_h/2), pad_h - math.ceil(pad_h/2)), mode='constant', value=0)
        padded_tensor = torch.nn.functional.pad(tensor, (0, pad_w, 0, pad_h), mode='constant', value=0)
        return padded_tensor

    def predict(self, prior, process):
        z_1 = fft.compute_moments_lnz(prior)
        z_2 = fft.compute_moments_lnz(process)
        prob_1 = prior/z_1
        prob_2 = process/z_2
        density_convolve = self.convolve(prob_1, prob_2)
        f, f1c, f1p, f2, f2f, eta = fft.analyze(torch.log(density_convolve + 1e-8))

        return eta, density_convolve

    def convolve(self, prob_1, prob_2):
        # padded_length = (2*self.band_limit[0] - 1,2*self.band_limit[1] - 1)
        # prob_1 = self.pad_for_fft_2d(prob_1, padded_length)
        # prob_2 = self.pad_for_fft_2d(prob_2, padded_length)
        moments_1, l_n_z_1 = fft.compute_moments_lnz(prob_1)
        moments_2, l_n_z_2 = fft.compute_moments_lnz(prob_2)
#         moments_1 = torch.fft.fftshift(torch.fft.fft2(prob_1),dim=(1,2))
#         moments_2 = torch.fft.fftshift(torch.fft.fft2(prob_2),dim=(1,2))
        moments_convolve = moments_1 * moments_2
        unnorm_density_convolve, f1c, f1p, f2, f2f, f_hat = fft.synthesize(moments_convolve)
        # unnorm_density_convolve_final = unnorm_density_convolve[:,math.floor(self.band_limit[0]/2):self.band_limit[0] +  math.floor(self.band_limit[0]/2),math.floor(self.band_limit[1]/2):self.band_limit[1] +  math.floor(self.band_limit[1]/2)].real
        # unnorm_density_convolve_final = unnorm_density_convolve[:,:self.band_limit[0],:self.band_limit[1]].real
        # unnorm_density_convolve = torch.clamp(unnorm_density_convolve,min=1e-8)
        # z_3 = self.normalisation_constant(unnorm_density_convolve)
        z_3 = ((moments_convolve[:,self.band_limit[0]//2,self.band_limit[1]//2, self.band_limit[2]//2].real)).unsqueeze(-1).unsqueeze(-1)
        
        density_convolve = abs(unnorm_density_convolve)/z_3

        return density_convolve

    def convert_from_energy_eta(self,density_samples):
        # z = self.normalisation_constant(density_samples)
        # density_samples_norm = density_samples/z
        energy_samples = torch.log(density_samples + 1e-10)
        f, f1c, f1p, f2, f2f, eta = fft.analyze(energy_samples)
        return eta

    def convert_from_eta_energy(self, eta):
        energy , f1c, f1p, f2, f2f, f_hat = fft.synthesize(eta)
        prob_unnorm = torch.exp(energy.real)
        moments_1, l_n_z_1 = fft.compute_moments_lnz(prob_unnorm)
        density = prob_unnorm/torch.exp(l_n_z_1)
        return density.real

    def update(self,predict, measurements):
        eta = predict + self.convert_from_energy_eta(measurements)
        return self.convert_from_eta_energy(eta)

#     def normalisation_constant(self,density):
#         # maximum = torch.max(energy, dim=-1).values.unsqueeze(-1)
#         moments,  = fft.compute_moments_lnz(prob_unnorm)
#         z_ = torch.real((moments[:,self.band_limit[0]//2,self.band_limit[1]//2]*(self.range_x_diff*self.range_y_diff))/math.prod(self.band_limit)).unsqueeze(-1).unsqueeze(-1)
#         return z_

    def analytic_filter(self,control,motion_noise,measurements,measurement_noise,prior_pdf):
        process = GaussianSE2(control, motion_noise, self.band_limit)
        process_pdf  = process.density()
        eta_bel_x_t_bar, density_bel_x_t_bar = self.predict(prior_pdf, process_pdf)
#         density_bel_x_t_bar = density_bel_x_t_bar.to(torch.float32)
        measurement_likelihood = GaussianSE2(measurements,measurement_noise,self.band_limit)
        measurement_pdf = measurement_likelihood.density()    
        posterior_pdf = self.update(eta_bel_x_t_bar, measurement_pdf)
#         posterior_pdf = posterior_pdf.to(torch.float32)
        return posterior_pdf, measurement_pdf,density_bel_x_t_bar

In [4]:
# Test the FFT

import torch
from typing import Tuple

def se2_grid_samples_torch(
    batch_size,
    size: Tuple[int, int, int] = (5, 5, 5),
    lower_bound: float = -0.5,
    upper_bound: float = 0.5
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Generate grid samples in SE(2) space.

    Args:
        size (Tuple[int, int, int]): Grid size for (x, y, theta) dimensions.
        lower_bound (float): Lower bound for x and y dimensions.
        upper_bound (float): Upper bound for x and y dimensions.

    Returns:
        Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
            - poses: Flattened grid of SE(2) poses (N, 3), where N is the total number of grid points.
            - X: Meshgrid tensor for the x dimension.
            - Y: Meshgrid tensor for the y dimension.
            - T: Meshgrid tensor for the theta dimension.
    """
    xs = torch.linspace(lower_bound, upper_bound, size[0]+1)[:-1]
    ys = torch.linspace(lower_bound, upper_bound, size[1]+1)[:-1]
    ts = torch.linspace(-torch.pi, torch.pi, size[2]+1)[:-1]
    

    X, Y, T = torch.meshgrid(xs, ys, ts, indexing='ij')

    # Flatten the grid and stack into poses
    poses = torch.stack((X.flatten(), Y.flatten(), T.flatten()), dim=-1)
    
    poses_batch = poses.unsqueeze(0).repeat((batch_size,1, 1))

    return poses_batch, X, Y, T


In [5]:
grid_size = (50, 50,100)
batch_size = 32
poses, X, Y, T = se2_grid_samples_torch(batch_size ,grid_size)

mu = torch.tensor([0.1, -0.1,  math.pi/2 ]).unsqueeze(0).repeat(batch_size,1,1)
cov = torch.diag(torch.tensor([0.6, 0.6, 0.6]))
distribution = GaussianSE2(mu, cov, grid_size)
density = distribution.density(poses)
density = density.reshape((batch_size, *grid_size))
energy = distribution.energy(poses)
energy = energy.reshape((batch_size, *grid_size))

In [8]:
fft= SE2_FFT(spatial_grid_size=grid_size,
                      interpolation_method='spline',
                      spline_order=1, 
                      oversampling_factor=1)
value = poses[:, 0, :]
d = fft.neg_log_likelihood(energy,value)
print(d)
# energy_true =  distribution.energy(value)
# print(energy_true.squeeze(0))
density_true =  distribution.density(value)
print(torch.log(density_true).squeeze(0))
nc = fft.compute_moments_lnz(energy)
print(nc)

# _ , _, _, _, _, eta = fft.analyze(energy)
# negative_energy, _, _, _, _, _ = fft.synthesize(eta)


tensor([4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114,
        4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114,
        4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114, 4.7114,
        4.7114, 4.7114, 4.7114, 4.7114, 4.7114])
tensor([[-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        ...,
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801]])
tensor([2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799,
        2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799,
        2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799, 2.3799,
        2.3799, 2.3799, 2.3799, 2.379

In [9]:
4.7114 - 2.3799

2.3315

In [135]:
poses.shape

torch.Size([32, 50, 50, 100, 3])

In [7]:
import torch

def circular_pad(f, pad):
    """
    Apply circular padding to a 2D tensor.
    
    Args:
        f (torch.Tensor): Input tensor of shape (H, W).
        pad (tuple): Padding in the form (top, bottom, left, right).
    
    Returns:
        torch.Tensor: Circularly padded tensor.
    """
    top, bottom, left, right = pad

    # Pad vertically (top and bottom)
    f_padded = torch.cat([f[:, -top:], f, f[:, :bottom]], dim=1) if top or bottom else f

    # Pad horizontally (left and right)
    f_padded = torch.cat([f_padded[:, :, -left:], f_padded, f_padded[:, :, :right]], dim=2) if left or right else f_padded

    return f_padded

def bilinear_interpolate_torch_circular_padded(f, x, y):
    """
    Perform bilinear interpolation on a 2D tensor with circular boundary conditions using padding.
    
    Args:
        f (torch.Tensor): Input tensor of shape (H, W).
        x (torch.Tensor): x-coordinates to interpolate (float tensor).
        y (torch.Tensor): y-coordinates to interpolate (float tensor).
    
    Returns:
        torch.Tensor: Interpolated values at the given coordinates.
    """
    B, H, W = f.shape
    pad = (1, 1, 1, 1)  # (top, bottom, left, right)

    # Circularly pad the tensor
    f_padded = circular_pad(f, pad)

    # Adjust coordinates for the padded tensor
    x = x + 1  # Shift by the padding
    y = y + 1

    # Get integer grid points
    x0 = torch.floor(x).long()
    x1 = x0 + 1
    y0 = torch.floor(y).long()
    y1 = y0 + 1

    # Gather values from the padded tensor
    Ia = f_padded[:, x0, y0]  # Bottom-left
    Ib = f_padded[:, x1, y0]  # Top-left
    Ic = f_padded[:, x0, y1]  # Bottom-right
    Id = f_padded[:, x1, y1]  # Top-right

    # Compute weights for interpolation
    wa = (x1 - x) * (y1 - y)  # Weight for Ia
    wb = (x1 - x) * (y - y0)  # Weight for Ib
    wc = (x - x0) * (y1 - y)  # Weight for Ic
    wd = (x - x0) * (y - y0)  # Weight for Id

    # Compute the interpolated result
    result = wa * Ia + wb * Ib + wc * Ic + wd * Id

    return result


In [8]:
import torch

def bilinear_interpolate_torch_with_nan(f, x, y):
    """
    Perform bilinear interpolation on a 2D tensor with `NaN` padding in PyTorch.
    
    Parameters:
        f (torch.Tensor): Input tensor of shape (H, W).
        x (torch.Tensor): x-coordinates to interpolate (float tensor).
        y (torch.Tensor): y-coordinates to interpolate (float tensor).
    
    Returns:
        torch.Tensor: Interpolated values at the given coordinates.
    """

    # Adjust coordinates for the padded tensor
    x = x + 1
    y = y + 1

    # Get integer grid points
    x0 = torch.floor(x).long()
    x1 = x0 + 1
    y0 = torch.floor(y).long()
    y1 = y0 + 1

#     pdb.set_trace()
    # Gather values from the 2D tensor
    Ia = f[:, x0, y0]  # Bottom-left
    Ib = f[:, x1, y0]  # Top-left
    Ic = f[:, x0, y1]  # Bottom-right
    Id = f[:, x1, y1]  # Top-right
    
#     print(f"Ia {Ia}, Ib {Ib}, Ic {Ic}, Id {Id}")
    # Compute the weights for interpolation
    wa = (x1 - x) * (y1 - y)  # Weight for Ia
    wb = (x1 - x) * (y - y0)  # Weight for Ib
    wc = (x - x0) * (y1 - y)  # Weight for Ic
    wd = (x - x0) * (y - y0)  # Weight for Id
    
#     print(f"wa {wa}, wb {wb}, wc {wc}, wd {wd}")

    # Mask out NaN values
    valid_a = ~torch.isnan(Ia)
    valid_b = ~torch.isnan(Ib)
    valid_c = ~torch.isnan(Ic)
    valid_d = ~torch.isnan(Id)
    
#     print(f"valid_a {valid_a}, valid_b {valid_b}, valid_c {valid_c}, valid_d {valid_d}")
    

    # Set weights to zero where values are NaN
    wa = wa * valid_a
    wb = wb * valid_b
    wc = wc * valid_c
    wd = wd * valid_d

    # Normalize weights to handle missing data
    weight_sum = wa + wb + wc + wd
#     print("weight_sum", weight_sum)
    
    Ia = torch.where(valid_a, Ia, torch.zeros_like(Ia))
    Ib = torch.where(valid_b, Ib, torch.zeros_like(Ib))
    Ic = torch.where(valid_c, Ic, torch.zeros_like(Ic))
    Id = torch.where(valid_d, Id, torch.zeros_like(Id))
    
#     wa = wa / (weight_sum + 1e-8)
#     wb = wb / (weight_sum + 1e-8)
#     wc = wc / (weight_sum + 1e-8)
#     wd = wd / (weight_sum + 1e-8)

    # Compute the interpolated value
    result = wa * Ia + wb * Ib + wc  * Ic + wd  * Id
#     print("result",result)
    result[weight_sum == 0] = float('nan')  # If all weights are zero, output NaN

    return result


In [15]:
import torch
import torch.fft as fft
import torch.nn.functional as F
from src.distributions.SE2.T1FFT import T1FFT
from src.distributions.SE2.T2FFT import T2FFT
from einops import rearrange
import math

import pdb

def preprocess_input_with_cval(input, cval=float('nan')):
    """
    Extend the input tensor with a border filled with a constant value.
    
    Parameters:
        input (torch.Tensor): Input tensor of shape (N, C, H, W).
        cval (float): The constant value to use for the extended border.
    
    Returns:
        torch.Tensor: Extended input tensor with out-of-bounds area filled with cval.
    """
    N, C, H, W = input.shape
    
    # Create an extended tensor with NaN-filled border
    extended = torch.full((N, C, H + 2, W + 2), cval, dtype=input.dtype, device=input.device)
    extended[:, :, 1:-1, 1:-1] = input  # Copy the original input into the center
    
    return extended

def adjust_grid_for_extension(grid):
    """
    Adjust the grid to account for the extended input tensor.
    
    Parameters:
        grid (torch.Tensor): Sampling grid of shape (N, H_out, W_out, 2).
    
    Returns:
        torch.Tensor: Adjusted grid of the same shape.
    """
    # Adjust grid to account for the one-pixel extension on all sides
    
    N, H, W, C = grid.shape
    adjusted_grid = grid.clone()
#     adjusted_grid[..., 0] = adjusted_grid[..., 0] * (H / (H + 10))
#     adjusted_grid[..., 1] = adjusted_grid[..., 1] * (W / (W + 10))
#     N, H, W, C = adjusted_grid.shape
    adjusted_grid[..., 0] = 2.0 * adjusted_grid[..., 0] / (W - 1) - 1.0  # Normalize x
    adjusted_grid[..., 1] = 2.0 * adjusted_grid[..., 1] / (H - 1) - 1.0  # Normalize y
    
    return adjusted_grid

def map_coordinates_torch_circular(input_tensor, coords):
    """
    Mimic scipy.ndimage.map_coordinates using PyTorch.
    :param input_tensor: Input tensor of shape (H, W).
    :param coords: Coordinates for sampling, shape (2, N) where N is the number of points.
    :param order: Interpolation order (only supports 1 for bilinear interpolation).
    :param mode: Boundary mode ('constant', 'nearest', 'reflect', 'wrap').
    :param cval: Constant value for 'constant' mode.
    :return: Interpolated values at the given coordinates.
    """

    fa = torch.empty((input_tensor.shape[0], input_tensor.shape[1] + 1, input_tensor.shape[2] + 1))
    fa[:, :-1, :-1] = input_tensor
    fa[:, -1, :-1] = input_tensor[:, 0, :]
    fa[:, :-1, -1] = input_tensor[:, :, 0]
    fa[:, -1, -1] = input_tensor[:, 0, 0]
    
#     fa = fa.unsqueeze(0)

    wrapped_coords_x = coords[0, ...] % input_tensor.shape[1]
    wrapped_coords_y = coords[1, ...] % input_tensor.shape[2]
    
#     hp = abs(fa.shape[1] - wrapped_coords_x.shape[1])
#     vp = abs(fa.shape[2] - wrapped_coords_x.shape[2])
    
    
                    
    
#     f = torch.nn.functional.pad(fa, (1, 1, 1, 1), mode='circular')
        
    sampled = bilinear_interpolate_torch_circular_padded(fa, wrapped_coords_x ,wrapped_coords_y)
    return sampled


def map_coordinates_torch_constant(input_tensor, coords):
    """
    Mimic scipy.ndimage.map_coordinates using PyTorch.
    :param input_tensor: Input tensor of shape (H, W).
    :param coords: Coordinates for sampling, shape (2, N) where N is the number of points.
    :param order: Interpolation order (only supports 1 for bilinear interpolation).
    :param mode: Boundary mode ('constant', 'nearest', 'reflect', 'wrap').
    :param cval: Constant value for 'constant' mode.
    :return: Interpolated values at the given coordinates.
    """
#     input_tensor = input_tensor.unsqueeze(0).unsqueeze(0)
#     fa = preprocess_input_with_cval(input_tensor, cval=cval)

    fa = torch.empty((input_tensor.shape[0],input_tensor.shape[1] + 1, input_tensor.shape[2] + 1))
    fa[:, :-1, :-1] = input_tensor
    fa[:, -1, :-1] = input_tensor[:, 0, :]
    fa[:, :-1, -1] = input_tensor[:, :, 0]
    fa[:, -1, -1] = input_tensor[:, 0, 0]
    
    wrapped_coords_x = coords[0, ...] % input_tensor.shape[1]
    wrapped_coords_y = coords[1, ...] % input_tensor.shape[2]
    
    f = torch.nn.functional.pad(fa,(1, 1, 1, 1),mode='constant', value=float('nan'))

    sampled = bilinear_interpolate_torch_with_nan(f, wrapped_coords_x ,wrapped_coords_y)
    return sampled

def shift_fft(f):
    nx = f.shape[1]
    ny = f.shape[2]
    p0, q0 = nx // 2, ny // 2

    X, Y = torch.meshgrid(
        torch.arange(p0, p0 + nx) % nx,
        torch.arange(q0, q0 + ny) % ny,
        indexing="ij",
    )

    fs = f[:,X, Y,...]
    return T2FFT.analyze(fs, axes=(1, 2))

def shift_ifft(fh):
    nx = fh.shape[1]
    ny = fh.shape[2]
    p0, q0 = nx // 2, ny // 2

    X, Y = torch.meshgrid(
        torch.arange(-p0, -p0 + nx,dtype=int) % nx,
        torch.arange(-q0, -q0 + ny,dtype=int) % ny,
        indexing="ij",
    )

    fs = T2FFT.synthesize(fh, axes=(1, 2))
    f = fs[:, X, Y,...]
    return f


class SE2_FFT_torch():
    def __init__(
        self,
        spatial_grid_size,
        interpolation_method="spline",
        spline_order=1,
        oversampling_factor=1,
    ):
        self.spatial_grid_size = spatial_grid_size  # tau_x, tau_y, theta
        self.interpolation_method = interpolation_method

        if interpolation_method == "spline":
            self.spline_order = spline_order

            self.p0 = spatial_grid_size[0] // 2
            self.q0 = spatial_grid_size[1] // 2
            self.r_max = torch.sqrt(torch.tensor(self.p0**2 + self.q0**2))

            self.n_samples_r = int(oversampling_factor * (torch.ceil(self.r_max) + 1))
            self.n_samples_t = int(
                oversampling_factor * (torch.ceil(2 * torch.pi * self.r_max))
            )

            r = torch.linspace(0.0, self.r_max, self.n_samples_r)
            theta = torch.linspace(-torch.pi, torch.pi, self.n_samples_t + 1)[:-1]
            R, THETA = torch.meshgrid(r, theta, indexing="ij")

            X = R * torch.cos(THETA)
            Y = R * torch.sin(THETA)

            I = X + self.p0
            J = Y  + self.q0
            self.c2p_coords = torch.stack((I, J), dim=0)

            i = torch.arange(0, self.spatial_grid_size[0])
            j = torch.arange(0, self.spatial_grid_size[1])
            x = i  - self.p0
            y = j - self.q0
            X, Y = torch.meshgrid(x, y, indexing="ij")

            R = torch.sqrt(X**2 + Y**2)
            T = torch.atan2(Y, X)

#             R *= (self.n_samples_r - 1) / self.r_max
#             T *= self.n_samples_t / (2 * torch.pi)

            self.p2c_coords = torch.stack((R, T), dim=0)

        elif interpolation_method == "Fourier":
            r_max = 1.0 / torch.sqrt(torch.tensor(2.0))
            nr = 15 * torch.ceil(r_max * spatial_grid_size[0])
            nt = 5 * torch.ceil(2 * torch.pi * r_max * spatial_grid_size[0])
            nx, ny = spatial_grid_size[:2]
            self.flerp = FourierInterpolator.init_cartesian_to_polar(nr, nt, nx, ny)
        else:
            raise ValueError("Unknown interpolation method:" + str(interpolation_method))

    def analyze(self, f):
        f1c = shift_fft(f)
        f1p = self.resample_c2p_3d(f1c)

        f2 = T1FFT.analyze(f1p.conj(), axis=3).conj()
        m_min = -math.floor(f2.shape[3] / 2.0)
        m_max = math.ceil(f2.shape[3] / 2.0) - 1
        varphi = torch.linspace(-torch.pi, torch.pi, f2.shape[2]+ 1)[:-1]
        factor = torch.exp(-1j * varphi[None, :, None] * torch.arange(m_min, m_max+1)[None, None, :]).unsqueeze(0)
        f2f = f2 * factor
        f_hat = T1FFT.analyze(f2f.conj(), axis=2).conj()
        return f, f1c, f1p, f2, f2f, f_hat

    def synthesize(self, f_hat):
        f2f = T1FFT.synthesize(f_hat.conj(), axis=2).conj()
        m_min = -math.floor(f2f.shape[3] / 2.0)
        m_max = math.ceil(f2f.shape[3] / 2.0) - 1
        psi = torch.linspace(-torch.pi, torch.pi, f2f.shape[2]+1)[:-1]
        factor = torch.exp(1j * psi[:, None] * torch.arange(m_min, m_max + 1)[None, :])
        f2 = f2f * factor[None,None, ...]
        f1p = T1FFT.synthesize(f2.conj(), axis=3).conj()
        f1c = self.resample_p2c_3d(f1p)
        f = shift_ifft(f1c)
        return f, f1c, f1p, f2, f2f, f_hat

    def resample_c2p(self, fc):
        """
        Resample a function on a Cartesian grid to a polar grid.
        :param fc: function values sampled on a Cartesian grid.
        :return: resampled function on a polar grid
        """
        fp_r = map_coordinates_torch_constant(fc.real, self.c2p_coords)
        fp_c = map_coordinates_torch_constant(fc.imag, self.c2p_coords)
        fp = fp_r + 1j * fp_c
        return fp

    def resample_p2c(self, fp):
        """
        Resample a function on a polar grid to a Cartesian grid.
        :param fp: function values sampled on a polar grid.
        :return: resampled function on a Cartesian grid
        """
        fc_r = map_coordinates_torch_circular(fp.real,self.p2c_coords)
        fc_c = map_coordinates_torch_circular(fp.imag,self.p2c_coords)
        fc = fc_r + 1j * fc_c
        return fc

    def resample_c2p_3d(self, fc):
        """
        Resample a 3D function on a Cartesian grid to a polar grid.
        :param fc: 3D function values sampled on a Cartesian grid.
        :return: resampled function on a polar grid
        """
        if self.interpolation_method == 'spline':
            fp = [self.resample_c2p(fc[:, :, :, i]) for i in range(fc.shape[3])]
            return torch.stack(fp, dim=-1)

        elif self.interpolation_method == 'Fourier':
            fp = [self.flerp.forward(fc[:, :, i]) for i in range(fc.shape[2])]
            return torch.stack(fp, dim=-1)

    def resample_p2c_3d(self, fp):
        """
        Resample a 3D function on a polar grid to a Cartesian grid.
        :param fp: 3D function values sampled on a polar grid.
        :return: resampled function on a Cartesian grid
        """
        if self.interpolation_method == 'spline':
            fc = [self.resample_p2c(fp[:, :, :, i]) for i in range(fp.shape[3])]
            return torch.stack(fc, dim=-1)

        elif self.interpolation_method == 'Fourier':
            fc = [self.flerp.backward(fp[:, :, i]) for i in range(fp.shape[2])]
            return torch.stack(fc, dim=-1)

    def neg_log_likelihood(self, energy, pose: torch.Tensor) -> torch.Tensor:
        """
        Compute point-wise synthesize the SE2 Fourier transform M at a given pose.
        Args:
            eta (torch.Tensor): Fourier coefficients (eta) of SE2 distribution with shape [n, 3] where n is the number of
                                samples
            l_n_z (float): Log of normalization constant of SE2 distribution
            pose (torch.Tensor): Pose at which to interpolate the SE2 Fourier transform
            se2_fft: Object class for SE2 Fourier transform

        Returns:
            Probability of distribution determined by Fourier coefficients (moments) at given pose
        """
        l_n_z = self.compute_moments_lnz(energy)
        _, _, _, _, _, eta = self.analyze(energy)
        b_x, b_y, b_t = self.spatial_grid_size
        # Reshape in case single pose is provided
        if pose.ndim < 2:
            pose = rearrange(pose, "b -> 1 b")
        # Arrange pose samples in broadcastable shape
        dx, dy = rearrange(pose[:, 0] + 0.5, "b -> b 1 1 "), rearrange(pose[:, 1] + 0.5, "b -> b 1 1")
        d_theta = rearrange(pose[:, 2] + math.pi, "b -> b 1 1 1 1")
        # Synthesize signal to obtain first FFT
        _, _, _, f_p_psi_m, _, _ = self.synthesize(eta)
        # Shift the signal to the origin
        f_p_psi_m = rearrange(torch.fft.ifftshift(f_p_psi_m, dim=2), "b p n m -> b p n m 1")
        
        omega_n =  rearrange(torch.arange(b_t), "n -> 1 1 1 n 1")
        # Compute the value of f(x) using the inverse Fourier transform
        f_p_psi = torch.sum(f_p_psi_m.conj() * torch.exp(1j * omega_n * d_theta), dim=3).conj()
        print(f_p_psi.shape)
        # Map from polar to Cartesian grid
        f_p_p = self.resample_p2c_3d(f_p_psi)
        print(f_p_p.shape)
        # Finally, 2D inverse FFT
        f_p_p = torch.fft.ifftshift(f_p_p,dim=(1,2,3))

        t_x, t_y = 1, 1
        # Compute complex term
        angle_x = (
            1j * 2 * torch.pi * (1 / t_x) * rearrange(torch.arange(b_x), "nx -> 1 nx 1 ") * (dx)
        )  # Angle component in X
        angle_y = (
            1j * 2 * torch.pi * (1 / t_y) * rearrange(torch.arange(b_y), "ny -> 1 ny 1 ") * (dy)
        )  # Angle component in Y
        
        angle = rearrange(angle_x, "b nx 1 -> b nx 1 1") + rearrange(
            angle_y, "b ny 1 -> b 1 ny 1"
        )
        print(angle.shape)
        f = -torch.sum(f_p_p * torch.exp(angle), dim=(1, 2, 3)).real 

        return f
    
    def compute_moments_lnz(self, negative_energy):
        """
        Compute the moments of the distribution
        :param eta: canonical parameters in log prob space of distribution
        :param update: whether to update the moments of the distribution and log partition constant
        :return: moments and log partition constant
        """
        _ , _, _, _, _, eta = self.analyze(negative_energy)
        negative_energy, _, _, _, _, _ = self.synthesize(eta)
#         maximum = torch.max(torch.abs(negative_energy))
        _, _, _, _, _, unnormalized_moments = self.analyze(torch.exp(negative_energy))
        z_0 = 0
        z_1 = unnormalized_moments.shape[2] // 2
        z_2 = unnormalized_moments.shape[3] // 2
        constant = unnormalized_moments[:, z_0, z_1, z_2] * (math.pi * 2* 2* math.pi)
        l_n_z = torch.log(constant.real)
        # Update moments of distribution and constant only when needed
        return l_n_z.real

In [202]:
f1c = shift_fft(energy)

t2 tensor(2500.)


In [16]:
fft_check_1 = SE2_FFT_torch(spatial_grid_size=grid_size,
                      interpolation_method='spline',
                      spline_order=1, 
                      oversampling_factor=1)
value = poses[:, 0, :]
d = fft_check_1.neg_log_likelihood(energy,value)
print(d)
energy_true =  distribution.energy(value)
print(energy_true.squeeze(-1))
density_true =  distribution.density(value)
print(torch.log(density_true))
# nc = fft_check_1.compute_moments_lnz(energy)
# print(nc)

# _ , _, _, _, _, eta = fft_check_1.analyze(energy)
# negative_energy, _, _, _, _, _ = fft_check_1.synthesize(eta)


torch.Size([32, 37, 223, 1])
torch.Size([32, 50, 50, 1])
torch.Size([32, 50, 50, 1])
tensor([2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314,
        2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314,
        2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314, 2.3314,
        2.3314, 2.3314, 2.3314, 2.3314, 2.3314])
tensor([[-2.4895, -2.4895, -2.4895,  ..., -2.4895, -2.4895, -2.4895],
        [-2.4895, -2.4895, -2.4895,  ..., -2.4895, -2.4895, -2.4895],
        [-2.4895, -2.4895, -2.4895,  ..., -2.4895, -2.4895, -2.4895],
        ...,
        [-2.4895, -2.4895, -2.4895,  ..., -2.4895, -2.4895, -2.4895],
        [-2.4895, -2.4895, -2.4895,  ..., -2.4895, -2.4895, -2.4895],
        [-2.4895, -2.4895, -2.4895,  ..., -2.4895, -2.4895, -2.4895]])
tensor([[-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -4.4801, -4.4801],
        [-4.4801, -4.4801, -4.4801,  ..., -4.4801, -

In [215]:
energy_true.shape

torch.Size([32, 1])

In [208]:
d.shape

torch.Size([32])

In [187]:
negative_energy.shape

torch.Size([32, 50, 50, 100])

In [204]:
torch.mean(torch.abs(negative_energy - energy))

tensor(0.1742)

In [166]:
torch.mean(torch.abs(negative_energy - energy[0]))

tensor(0.1742)

In [38]:
poses[50].shape

torch.Size([3])

In [96]:
distribution.det_cov

tensor(0.0640)

In [53]:
energy_true =  distribution.energy(poses[45].unsqueeze(0))
print(energy_true)

tensor([-2.6519])


In [8]:
torch.log(distribution.normalizing_constant), distribution.normalizing_constant, 

(tensor(1.3824), tensor(3.9844))

In [102]:
torch.where(torch.logical_and(fft_check_1.p2c_coords[0] == float(35.3553),fft_check_1.p2c_coords[1] == 0.7854)))

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [98]:
fft_check_1.p2c_coords

tensor([[[35.3553, 34.6554, 33.9706,  ..., 33.3017, 33.9706, 34.6554],
         [34.6554, 33.9411, 33.2415,  ..., 32.5576, 33.2415, 33.9411],
         [33.9706, 33.2415, 32.5269,  ..., 31.8277, 32.5269, 33.2415],
         ...,
         [33.3017, 32.5576, 31.8277,  ..., 31.1127, 31.8277, 32.5576],
         [33.9706, 33.2415, 32.5269,  ..., 31.8277, 32.5269, 33.2415],
         [34.6554, 33.9411, 33.2415,  ..., 32.5576, 33.2415, 33.9411]],

        [[-2.3562, -2.3766, -2.3978,  ...,  2.4199,  2.3978,  2.3766],
         [-2.3358, -2.3562, -2.3775,  ...,  2.3996,  2.3775,  2.3562],
         [-2.3146, -2.3349, -2.3562,  ...,  2.3784,  2.3562,  2.3349],
         ...,
         [-0.8491, -0.8288, -0.8076,  ...,  0.7854,  0.8076,  0.8288],
         [-0.8270, -0.8067, -0.7854,  ...,  0.7632,  0.7854,  0.8067],
         [-0.8058, -0.7854, -0.7641,  ...,  0.7419,  0.7641,  0.7854]]])

In [29]:
2.9093/0.4980

5.841967871485944

In [33]:
14.7948/3.9844

3.713181407489208

In [36]:
3.9844/1.1773

3.384354030408562

In [51]:
7.3198/6.3713

1.1488707171220944

In [14]:
 a , b, c, d, e, eta = fft_check_1.analyze(energy)
negative_energy, f, g, x, h, i = fft_check_1.synthesize(eta)
maximum = torch.max(torch.abs(negative_energy))
_, _, _, _, _, unnormalized_moments = fft_check_1.analyze(torch.exp(negative_energy-maximum))

t2 tensor(2500.)
t1 32
t1 223
tensor(2500.)
t2 tensor(2500.)
t1 32
t1 223


In [24]:
i.shape

torch.Size([37, 223, 32])

In [14]:
temp = fft_check_1.resample_p2c_3d(unnormalized_moments)

In [37]:
import numpy as np
math.atan2(25, 25)

0.7853981633974483

In [80]:
max_value, flat_index = torch.max(torch.abs(temp.view(-1)), dim=0)

shape = temp.shape
z = flat_index // (shape[1] * shape[2])
y = (flat_index % (shape[1] * shape[2])) // shape[2]
x = flat_index % shape[2]

max_index = (z.item(), y.item(), x.item())

print("Maximum Value:", max_value.item())
print("Index of Maximum Value:", max_index)

Maximum Value: 3.375641086100778e-11
Index of Maximum Value: (24, 27, 16)


In [82]:
torch.argmax(abs(unnormalized_moments[0,:,16]))

tensor(111)

In [100]:
torch.log(temp[24, 27, 16] * unnormalized_moments[0,111, 16]).real * (4 * math.pi * math.pi)

tensor(-1572.0892)

In [81]:
unnormalized_moments[0,111, 16]

tensor(1.5044e-07-1.3006e-09j)

In [ ]:

tensor1_padded = torch.nn.functional.pad(
    temp.unsqueeze(2).unsqueeze(3) , (0, 0, 0, 222, 0, 36, 0,0, 0, 0)  # Pad along the 3rd dimension
)
# Pad tensor2 to match the target shape (50, 50, 223, 32)
tensor2_padded = torch.nn.functional.pad(
   unnormalized_moments.unsqueeze(0).unsqueeze(1) , (0, 0, 0,0,0,0,0, 49, 0, 49)  # Pad along the 1st and 2nd dimensions
)

In [ ]:
tensor1_expanded = temp.unsqueeze(2).unsqueeze(3)  # Shape (50, 50, 1, 1, 32)
tensor1_expanded = tensor1_expanded.expand(-1, -1, 37, 223, -1)  # Shape (50, 50, 37, 223, 32)

# Step 2: Expand tensor2 to shape (50, 50, 37, 223, 32)
# Add new dimensions for the 0th and 1st dimensions (50, 50)
tensor2_expanded = unnormalized_moments.unsqueeze(0).unsqueeze(0)  # Shape (1, 1, 37, 223, 32)
tensor2_expanded = tensor2_expanded.expand(50, 50, -1, -1, -1)  # Shape (50, 50, 37, 223, 32)

# Step 3: Concatenate the two tensors along the 2nd dimension (dim=2)
final_tensor = torch.cat([tensor1_expanded, tensor2_expanded], dim=2)  # Shape (50, 50, 37, 223, 32)

print("Final Tensor Shape:", final_tensor.shape)  # Should be (50, 50, 37, 223, 32)

In [ ]:
import torch

# Example tensors
tensor1 = torch.randn(50, 50, 32)  # Shape (50, 50, 32)
tensor2 = torch.randn(37, 223, 32)  # Shape (37, 223, 32)

# Step 1: Expand tensor1 to shape (50, 50, 37, 223, 32)
# Add new dimensions for the 2nd and 3rd dimensions (37, 223)
tensor1_expanded = tensor1.unsqueeze(2).unsqueeze(3)  # Shape (50, 50, 1, 1, 32)
tensor1_expanded = tensor1_expanded.expand(-1, -1, 37, 223, -1)  # Shape (50, 50, 37, 223, 32)

# Step 2: Expand tensor2 to shape (50, 50, 37, 223, 32)
# Add new dimensions for the 0th and 1st dimensions (50, 50)
tensor2_expanded = tensor2.unsqueeze(0).unsqueeze(0)  # Shape (1, 1, 37, 223, 32)
tensor2_expanded = tensor2_expanded.expand(50, 50, -1, -1, -1)  # Shape (50, 50, 37, 223, 32)

# Step 3: Concatenate the two tensors over multiple dimensions (0, 1, 2, 3)
final_tensor = torch.cat([tensor1_expanded, tensor2_expanded], dim=2)  # Concatenate along dim=2

print("Final Tensor Shape:", final_tensor.shape)  # Should be (50, 50, 37, 223, 32)


In [2]:
import numpy as np

In [23]:
gt_trajectory_ = np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [3.0, 0.0, 0.0]])
trajectory_ = np.array([[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [3.0, 0.0, 0.0]])
scaling_factor = 1
offset_x = 0
offset_y = 0

In [24]:
trajectory.shape

(3, 4)

In [25]:
gt_trajectory = gt_trajectory_[:, :2].T
# Scale the gt trajectory
gt_trajectory[0, :] = (gt_trajectory[0, :] / scaling_factor) - offset_x
gt_trajectory[1, :] = (gt_trajectory[1, :] / scaling_factor) - offset_y
zeros = np.zeros((1, gt_trajectory.shape[1]))

# Append zeros in third dimension as z coordinate
gt_trajectory = np.vstack((gt_trajectory, zeros))

# Scale second trajectory
trajectory = trajectory_[:, :2].T
trajectory[0, :] = (trajectory[0, :] / scaling_factor) - offset_x
trajectory[1, :] = (trajectory[1, :] / scaling_factor) - offset_y
trajectory = np.vstack((trajectory, zeros))
# Align trajectory
rot, trans, trans_error = align(trajectory, gt_trajectory)
aligned_trajectory = rot @ trajectory + trans
# Compute metrics
np.sqrt(np.dot(trans_error, trans_error) / len(trans_error))



0.5

In [10]:
def align(model, data):
    """Align two trajectories using the method of Horn (closed-form).

    Input:
    model -- first trajectory (3xn)
    data -- second trajectory (3xn)

    Output:
    rot -- rotation matrix (3x3)
    trans -- translation vector (3x1)
    trans_error -- translational error per point (1xn)

    """
    np.set_printoptions(precision=3, suppress=True)
    model_zerocentered = model - model.mean(1).reshape(-1, 1)
    data_zerocentered = data - data.mean(1).reshape(-1, 1)

    W = np.zeros((3, 3))
    for column in range(model.shape[1]):
        W += np.outer(model_zerocentered[:, column], data_zerocentered[:, column])
    U, d, Vh = np.linalg.linalg.svd(W.transpose())
    S = np.array(np.identity(3))
    if (np.linalg.det(U) * np.linalg.det(Vh) < 0):
        S[2, 2] = -1
    rot = U @ S @ Vh
    trans = data.mean(1).reshape(-1, 1) - rot @ model.mean(1).reshape(-1, 1)

    model_aligned = rot @ model + trans
    #alignment_error = model_aligned - data
    alignment_error = model - data

    trans_error = np.sqrt(np.sum(alignment_error * alignment_error, 0))

    return rot, trans, trans_error

In [27]:
import torch

# Convert to torch tensors
gt_trajectory_ = torch.tensor([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [3.0, 0.0, 0.0]], dtype=torch.float32)
trajectory_ = torch.tensor([[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [3.0, 0.0, 0.0]], dtype=torch.float32)
scaling_factor = 1
offset_x = 0
offset_y = 0

In [33]:
import torch

# Convert to torch tensors
gt_trajectory = gt_trajectory_[:, :2].T
trajectory = trajectory_[:, :2].T

# Scale the gt trajectory
gt_trajectory[0, :] = (gt_trajectory[0, :] / scaling_factor) - offset_x
gt_trajectory[1, :] = (gt_trajectory[1, :] / scaling_factor) - offset_y
zeros = torch.zeros((1, gt_trajectory.shape[1]), dtype=torch.float32)

# Append zeros in third dimension as z coordinate
gt_trajectory = torch.vstack((gt_trajectory, zeros))

# Scale second trajectory
trajectory[0, :] = (trajectory[0, :] / scaling_factor) - offset_x
trajectory[1, :] = (trajectory[1, :] / scaling_factor) - offset_y
trajectory = torch.vstack((trajectory, zeros))

# Align trajectory
rot, trans, trans_error = align(trajectory, gt_trajectory)
rot = rot.to(torch.float32)
trans =trans.to(dtype=torch.float32)
aligned_trajectory = rot @ trajectory + trans

# Compute metrics
trans_error = trans_error.to(torch.float32)
metrics = torch.sqrt(torch.dot(trans_error, trans_error) / len(trans_error))

print(metrics)

tensor(0.5000)


In [30]:
import torch

def align(model, data):
    """Align two trajectories using the method of Horn (closed-form).

    Input:
    model -- first trajectory (3xn)
    data -- second trajectory (3xn)

    Output:
    rot -- rotation matrix (3x3)
    trans -- translation vector (3x1)
    trans_error -- translational error per point (1xn)

    """
    model_zerocentered = model - model.mean(1, keepdim=True)
    data_zerocentered = data - data.mean(1, keepdim=True)

    W = torch.zeros((3, 3), dtype=model.dtype, device=model.device)
    for column in range(model.shape[1]):
        W += torch.outer(model_zerocentered[:, column], data_zerocentered[:, column])
    U, d, Vh = torch.linalg.svd(W.T)
    S = torch.eye(3, dtype=model.dtype, device=model.device)
    if (torch.det(U) * torch.det(Vh) < 0):
        S[2, 2] = -1
    rot = U @ S @ Vh
    trans = data.mean(1, keepdim=True) - rot @ model.mean(1, keepdim=True)

    model_aligned = rot @ model + trans
    alignment_error = model - data

    trans_error = torch.sqrt(torch.sum(alignment_error * alignment_error, 0))

    return rot, trans, trans_error